# Homework 7: Southern Ocean Reentrant Channel

#### Objective:
In this homework, we will investigate the effect of resolution on transient features and transport quantities using the Southern Ocean Reentrant Channel model.

#### Import Modules:
Begin by importing the modules to read in data, conduct calculations, and make plots.

In [1]:
# import the modules you need here - you may need to add as you go
import os
import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cm
import moviepy.video.io.ImageSequenceClip

# this module will help organize your subplots
from matplotlib.gridspec import GridSpec

# this module will help create your elapsed time bar
from matplotlib.patches import Rectangle

## Part 1: Modify, Compile and Run the Model

In this assignment, we are going to run the same model we ran in class - but this time, we will run at a resolution which is 10x higher than before. Follow the steps below to get the high resolution model set up.

#### Step 1: Clean Build and Run directories
For this model, we're going to need some clean build and run directories. If you already downloaded everything from the low resolution models in class, you can just remove all of the existing files in your run and bulid directories. Alternatively, you can make new versions e.g.

```
MITgcm/verification/tutorial_reentrant_channel
mkdir build_hires
mkdir run_hires
```

If you choose this option, be sure to replace "build" and "run" in the lines below with these new directories.

#### Step 2: Recompile the model

To compile the model for higher resolution, we're going to need to resize our domain. The new domain size is stored in the `SIZE.h_eddy` file in the code directory. To make this change, move to your code directory and then rename the SIZE.h_eddy file to SIZE.h. Then, in your build directory, compile with MPI.

#### Step 3: Prepare your run directory
To start, per the usual set up, copy everything from input to your run directory and link the `mitgcmuv` from build to run.

Then, on your local machine, run the `gendata.5km.py` script provided with the model input files. This should create the following files:
- SST_relax.5km.bin
- T_relax_mask.5km.bin
- bathy.5km.bin
- temperature.5km.bin
- zonal_wind.5km.bin

In addition, grab the pickup files from Canvas provided with this notebook and upload them to Spartan in your run directory.
  
Make a copy of these files in your run directory on Spartan.

Next, modify the following data* files:

1. Modify the data.pkg file to uncomment the line useGMRedi=.FALSE.,

2. Modify the data file to un/comment the lines for "eddy-permitting run":
- 4 lines in PARM01 
- comment out line for viscAh=2000 in parm01
- deltaT, nTimesteps, monitorFreq in PARM03
- delX and delY in PARM04
- 4x files in PARM05
  
3. Modify the data file to start at iteration 3483648 and run for two years (1 year = 360 days).

4. Modify the data.rbcs file to use the new lines

5. Modify the data.diagnostics file to be the same as that shown in class

#### Run the model
To run the model, you'll need to make a new job script. Consider the following questions:
- How many CPUs (tasks) will you need?

100 CPUs

- Each node is 28 CPUs. How many nodes will you need? 

4 nodes

If you are in doubt, send Mike an email.

Note: this model run will take a couple hours!

## Part 2: Surface Conditions
Using your output from the high-resolution model produced by your new model run, make a movie comparing the low resolution model output to the high resolution output. The top row should show the low resolution output, as shown in the movie made in class, and the bottom row should show the fields at an identical timestep from the high resolution model. Be sure to resize and reorganize your plotting code from class to make a well-formatted movie.

In [2]:
hires_path = 'C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run'

In [3]:
# make a function to output the following grids:
# theta, etan, speed
def read_surface_conditions_from_file(file_path):
    surf_grid = np.fromfile(file_path, '>f4').reshape(4,400,200)
    theta = surf_grid[0,:,:]
    eta = surf_grid[1,:,:]
    uvel = surf_grid[2,:,:]
    vvel = surf_grid[3,:,:]

    return(theta, eta, uvel, vvel)

In [4]:
def iteration_number_to_dec_yr(iteration_number):

    timestep = 250
    y1 = 28
    y2 = 30
    x1 = (y1*360*86400)/timestep
    x2 = (y2*360*86400)/timestep
    
    x = iteration_number 

    # low res intervals
    # x1 = 870912
    # x2 = 933120

    y = ((y2-y1)/(x2-x1))*(x-x1) +y1
    return(y)

In [18]:
file.split('.')[2]

'meta'

In [20]:
surf_list = []
for file in os.listdir(os.path.join(hires_path,'Diags')):
        if file.split('_')[0] == 'surf' and file.split('.')[2] == 'data':
            surf_list.append(file)

surf_list[:5]

['surf_diags_daily.0003483994.data',
 'surf_diags_daily.0003484339.data',
 'surf_diags_daily.0003484685.data',
 'surf_diags_daily.0003485030.data',
 'surf_diags_daily.0003485376.data']

In [40]:
# Edit the coding cells here to make your movie
# make a test plot here
for file in surf_list:
    file_path = os.path.join(hires_path,'Diags',file)

    
    theta, eta, uvel, vvel = read_surface_conditions_from_file(file_path)
    # masked_uvel = np.ma.masked_where(hfacw[0,:,:] ==0,uvel_grid)
    
    iteration = file.split('.')[1]
    img_path = os.path.join(hires_path,'plots','surf_'+iteration+'.png')

    fig = plt.figure(figsize = (12,5))

    gs = GridSpec(6, 3, figure = fig)
    ax1 = fig.add_subplot(gs[0:5, 0])
    plt.pcolormesh(theta, cmap= cm.thermal, vmin = 0, vmax = 10)
    plt.colorbar(label = 'degC', orientation='horizontal')
    plt.title('Temperature')
    plt.ylabel('Model Rows')
    
    ax2 = fig.add_subplot(gs[0:5, 1])
    plt.pcolormesh(eta,vmin = -0.9, vmax = 0.9)
    plt.colorbar(label = 'm', orientation='horizontal')
    plt.title('Sea Surface Height')
    plt.xlabel('Model Columns')
    
    ax3 = fig.add_subplot(gs[0:5, 2])
    plt.pcolormesh(np.sqrt(uvel**2 + vvel**2),cmap = 'turbo', vmin = 0, vmax = 2)
    plt.colorbar(label = 'm/s', orientation='horizontal')
    plt.title('Speed')
    
    ax4 = fig.add_subplot(gs[-1,:])
    bar = Rectangle((28,0),
              iteration_number_to_dec_yr(int(iteration))-28,1,
              color = 'gray')
    ax4.add_patch(bar)
    plt.xlim([28, 30])
    plt.ylim([0, 1])
    plt.xlabel('Decimal Year')
    ax4.yaxis.set_tick_params(labelcolor='none')
    
    plt.tight_layout()
    
    plt.savefig(img_path)
    plt.close(fig)


In [41]:
img_list = []
for img in os.listdir(os.path.join(hires_path,'plots')):
    img_add = os.path.join(hires_path,'plots',img)
    img_list.append(img_add)

img_list[:5]

['C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\\plots\\surf_0003483994.png',
 'C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\\plots\\surf_0003484339.png',
 'C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\\plots\\surf_0003484685.png',
 'C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\\plots\\surf_0003485030.png',
 'C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\\plots\\surf_0003485376.png']

In [42]:
# set the frames per second
fps=30

# use the ImageSequenceClip module to set up the clip
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(img_list, fps=fps)

# write the video to a file
output_file = os.path.join(hires_path,'surf_hires.mp4')
clip.write_videofile(output_file)

Moviepy - Building video C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\surf_hires.mp4.
Moviepy - Writing video C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\surf_hires.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/Stephen/Desktop/MLML/MS 274 Ocean Modeling/Lab/Week 7/HW7/run\surf_hires.mp4


## Part 3: Mean Hydrography
Again using your output from the high-resolution model, create a plot that shows mean properties along the channel ridge. You plot should have 8 panels - the 4 from class in one row, and the 4 from the new model in the bottom row.

In [ ]:
# Edit the coding cells here to make your plot


## Part 4: Heat Flux Timeseries
You probably see where this one is going - in this section, re-create your timeseries plot from class for mean temperature and zonal velocity, and total heat flux on a transect across the ridge. Each of the three panels should now have 2 lines - one for the coarse resolution model and one for the high resolution model.

In [ ]:
# Edit the coding cells here to make your plot

## Part 5: Model Differences
In the markdown cell below, write a paragraph describing the differences in the model runs that resulted from changing the resolution. Some key questions to consider:
- What are the key changes to the model output?
- What causes the variabiliy in the higher resolution model that was not present in the lower resolution model? Note here that the external forcing conditions are identical!
- What is consistent between the model runs?
- How well do you think the low resolution model captures the features of the higher resolution model?
- What generalizations can we make about coarse resolution global ocean models based on this comparison?

*Edit this markdown cell*